### Walk through the basics of Apache Cassandra:<br><li>Creating a table <li>Inserting rows of data<li>Running a simple SQL query to validate the information. 

In [32]:
# Import the python wrapper for cassandra
import cassandra

In [33]:
# Connect to the cassandra server
from cassandra.cluster import Cluster
try:
    cluster = Cluster(["127.0.0.1"])
    session = cluster.connect()
except Exception as e:
    print(e)

In [24]:
# Create a keyspeace udacity
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}"""
)
except Exception as e:
    print(e)

In [34]:
# Connect to the keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)   

### Create a Song Library that contains a list of songs, including the song name, artist name, year, album it was from, and if it was a single. 

`song_title
artist_name
year
album_name
single`

<li> In apache cassandra, tables are created based on query in order to bring about efficient retrieval.
    
<li> Hence, we create a table to ad-hoc query below
    
`select * from songs WHERE year=1970 AND artist_name="The Beatles"`

#### To make the query work efficiently
<li> artist_name will be the partition key
<li> year and song_title would be the clustering columns to make the PRIMARY KEY UNIQUE
<li> <strong>Remember that duplicates are not allowed in cassandra

In [36]:
query = "CREATE TABLE IF NOT EXISTS song_library"
query = query + "(song_title text, artist_name text, year int, album_name text, single boolean ,\
                    PRIMARY KEY(year, artist_name, song_title))"

try:
    session.execute(query)
except Exception as e:
    print(e)

- Lets insert rows

First Row:  "Across The Universe", "The Beatles", "1970", "False", "Let It Be"

Second Row: "The Beatles", "Think For Yourself", "False", "1965", "Rubber Soul"

In [38]:
query = "INSERT INTO song_library (song_title, artist_name, year, album_name, single) \
        VALUES (%s, %s, %s, %s, %s)"

try:
    session.execute(query, ('Across The Universe', 'The Beatles', 1970, 'Let It Be', 'False'))
except Exception as e:
    print(e)

In [39]:
try:
    session.execute(query, ('Think For Yourself', 'The Beatles', 1965, 'Rubber Soul', 'False'))
except Exception as e:
    print(e)

In [40]:
# Validate that data was inserted in the table
query = "SELECT * FROM song_library"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.song_title, row.artist_name, row.year, row.album_name, row.single)

Think For Yourself The Beatles 1965 Rubber Soul False
Across The Universe The Beatles 1970 Let It Be False


In [41]:
# Validate the query

query = "SELECT * FROM song_library WHERE YEAR=1970 AND artist_name='The Beatles'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_title, row.artist_name, row.year, row.album_name, row.single)

Across The Universe The Beatles 1970 Let It Be False


In [35]:
#Lets drop the table

# rows = "DROP TABLE song_library"
# try:
#     rows = session.execute(query)
# except Exception as e:
#     print(e)

- Close cluster and session connection

In [42]:
session.shutdown()
cluster.shutdown()